In [1]:
from src.lobbyingDataPage import *
from src.lobbyingScraper import *

import pickle
with open("tests/testfiles/2005html.pkl", "rb") as f:
    html05 = pickle.load(f)
with open("tests/testfiles/2020html.pkl", "rb") as f:
    html20 = pickle.load(f)
test_htmls = html05.__add__(html20)

In [2]:
%%capture
pages = convert_html_list(test_htmls[::10])

In [5]:
[page for page in pages if 'Activities' in page.tables.keys()]


In [4]:
soup = bs(html, 'html.parser')

In [9]:
def query_page(soup, table_header, table_footer, string=""):
    if not string:
        string = soup.text

    query_string = fr"(?<={table_header}).*?(?={table_footer})"
    query = re.compile(query_string,re.DOTALL)
    query_results = re.findall(query, string)
    return query_results

def pull_table(query_result, columns):
        split_text=[line.strip() for line in query_result.split('\n') if line.strip()]
        divided_text = list(divide_chunks(split_text, len(columns)))
        table_df = pd.DataFrame(divided_text, columns=columns)
        return table_df

columns = ['House / Senate','Bill Number or Agency Name','Bill title or activity','Agent position','Amount','Direct business association']
header = r"".join(columns)
footer = r'\xa0\xa0\xa0\nTotal'
query_results = query_page(soup, r'Lobbyist: ', r' amount\n')
for query_result in query_results:
    activities_table=query_page(soup, header, footer, query_result)[0]
    activities_df = pull_table(activities_table, columns)
    activities_df['Client'] = query_results[0].split('\n')[0]
    activities_df['Lobbyist'] = query_results[0].split('\n')[2]
    print(activities_df)

  House / Senate Bill Number or Agency Name  \
0      Executive                      Other   
1     House Bill                          2   

                              Bill title or activity Agent position Amount  \
0  Efforts to grow and expand the Commonwealth's ...        Support  $0.00   
1  An Act making appropriations for the fiscal ye...        Support  $0.00   

  Direct business association                   Client Lobbyist  
0                         N/A  Hugh Richard Jones, III           
1                         N/A  Hugh Richard Jones, III           
  House / Senate Bill Number or Agency Name  \
0     House Bill                          2   
1      Executive                      Other   

                              Bill title or activity Agent position Amount  \
0  An Act making appropriations for the fiscal ye...        Support  $0.00   
1  Efforts to address the Commonwealth's educatio...        Support  $0.00   

  Direct business association                   

In [30]:
query_result

"Hugh Richard Jones, III\n\n\n\n\n\n\n\n\n\n\nClient:\nMassachusetts Business Alliance for Education\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHouse / SenateBill Number or Agency NameBill title or activityAgent positionAmountDirect business association\n\n\nExecutive\n\nOther \n\nEfforts to grow and expand the Commonwealth's early college/college-in-high-school programs. \n\nSupport\n\r\n                                                    $0.00\r\n                                                \nN/A \n\n\n\nHouse Bill\n\n2 \n\nAn Act making appropriations for the fiscal year 2021 for the maintenance of the departments, boards, commissions, institutions, and certain activities of the commonwealth, for interest, sinking fund, and serial bond requirements, and for certain permanent improvements \n\nSupport\n\r\n                                                    $0.00\r\n                                                \nN/A \n\n\n\xa0\xa0\xa0\nTotal"

In [29]:
header

'House / SenateBill Number or Agency NameBill Title or activityAgent positionAmountDirect business association'

In [25]:
header

'House/SenateBill Number or Agency NameBill Title or activityAgent positionAmountDirect business association'

In [21]:
query_string = fr"(?<={header}).*?(?={footer})"
query = re.compile(query_string,re.DOTALL)
query_results = re.findall(query, query_result)
query_results


[]

In [11]:
soup.text

"\n\n\n\n\n\n\n\n\r\n\tLobbyist Public Search\r\n\n\n\n\n\n  \n\n\n\n\nHome\nDirections\nContact Us\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLobbyist Public Search\n\n\n\n\n\n\n\n\n\r\n            Records prior to registration year 2005 are not available online. \r\n            \n\r\n            \r\n            Contact the Lobbyist Division here for assistance.\r\n        \nSearch data is collected from registration records and disclosure reports \r\n        submitted by lobbyists, lobbying entities, and their clients. Massachusetts General Law requires any party providing or retaining \r\n        lobbying services to register annually, and submit disclosure reports twice a year.  \r\n        \n\n\nYour comments are welcome:\xa0\r\n            submit feedback\n\n\n\n\n\n\n\n\n\n\n\nDisclosure reporting details\n\n\n\nBack to search detail summary\n\n\n\n\nLobbyist Entity disclosure reporting period:\r\n                        \xa0 \r\n                        07/01/2020 - 12/31/2020

In [10]:
[page for page in pages if 'Activities' in page.tables.keys()]

In [14]:
page.query_page(header, footer, query_result)

[]

In [5]:
individuals = [page for page in pages if 'Lobbyist Entity' not in page.soup.text]
entities = [page for page in pages if 'Lobbyist Entity' in page.soup.text]

In [3]:
[page for page in pages if 'Campaign Contributions' in page.tables.keys()]

In [ ]:
def get_campaign_contributions(page):
    columns = ['Date','Recipient name','Office sought','Amount']
    header = "DateRecipient nameOffice soughtAmount"
    footer = "\xa0\xa0Total contributions"
    query_string = fr"(?<={header}).*?(?={footer})"
    query = re.compile(query_string,re.DOTALL)
    query_results = re.findall(query, page.soup.text)
    if not query_results:
        return
    for query_result in query_results:
        split_text=[line.strip() for line in query_result.split('\n') if line.strip()]
        divided_text = list(divide_chunks(split_text, 4))
        contributions_df = pd.DataFrame(divided_text, columns=columns)
        page.update_table('Campaign Contributions', contributions_df)

In [5]:
page = pages[0]
columns = ['Date','Recipient name','Office sought','Amount']
header = r"DateRecipient nameOffice soughtAmount"
footer = r"\xa0\xa0Total contributions"
query_string = fr"(?<={header}).*?(?={footer})"
query = re.compile(query_string,re.DOTALL)
query_results = re.findall(query, page.soup.text)
print(bool(query_results))
for query_result in query_results:
        split_text=[line.strip() for line in query_result.split('\n') if line.strip()]
        divided_text = list(divide_chunks(split_text, 4))
        contributions_df = pd.DataFrame(divided_text, columns=columns)
        page.update_table('Campaign Contributions', contributions_df)

True


In [6]:
page.tables.keys()

dict_keys(['Headers', 'Activities', 'Campaign Contributions'])

In [ ]:
    

    

    # Implemented seperately for lobbyists and entities?? NOPE
    

In [14]:
entities[0].soup.text

'\n\n\n\n\n\n\n\n\r\n\tLobbyist Public Search\r\n\n\n\n\n\n  \n\n\n\n\nHome\nDirections\nContact Us\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLobbyist Public Search\n\n\n\n\n\n\n\n\n\r\n            Records prior to registration year 2005 are not available online. \r\n            \n\r\n            \r\n            Contact the Lobbyist Division here for assistance.\r\n        \nSearch data is collected from registration records and disclosure reports \r\n        submitted by lobbyists, lobbying entities, and their clients. Massachusetts General Law requires any party providing or retaining \r\n        lobbying services to register annually, and submit disclosure reports twice a year.  \r\n        \n\n\nYour comments are welcome:\xa0\r\n            submit feedback\n\n\n\n\n\n\n\n\n\n\n\nDisclosure reporting details\n\n\n\nBack to search detail summary\n\n\n\n\nLobbyist Entity disclosure reporting period:\r\n                        \xa0 \r\n                        01/01/2005 - 06/30/2005

In [4]:
for page in pages:
    if 'Campaign Contributions' in page.tables.keys():
        print('YAY')

In [20]:
header = "DateRecipient nameOffice soughtAmount"
footer = "\xa0\xa0Total contributions"
query_string = fr"(?<={header}).*?(?={footer})"
query = re.compile(query_string,re.DOTALL)
query_results = re.findall(query, pages[0].soup.text)

In [21]:
query_results

['\n\n4/4/05\nMitt Romney \n\nGovernor of Massachusetts \n\r\n                            $200.00\r\n                        \n\n5/24/05\nAndrea F. Nuciforo, Jr. \n\nMassachusetts Senate-Berkshire, Hampden and Hampsh \n\r\n                            $150.00\r\n                        \n\n5/3/05\nRonald Mariano \n\nMassachusetts House of Representatives-3rd Norfolk \n\r\n                            $150.00\r\n                        \n\n5/4/05\nBrian P. Lees \n\nMassachusetts Senate-1st Hampden and Hampshire \n\r\n                            $200.00\r\n                        \n\n']

In [ ]:
from IPython.display import HTML

HTML(str(pages[5].soup.html))

In [ ]:
# Header:
# Authorizing Officer name / Lobbyist name
# Title / NA
# Business name / Business name
# Address / Address
# City, state, zip code / Citym, state, zip code
# country / country
# NA / Agent Type

#TABLES:
# Entities
# Lobbyists
# Client Compensation
# Compensation/Salaries Paid
# Activities, Bill Numbers and Titles
# Operating Expenses
# Meals, Travel, and Entertainment Expenses
# Additional Expenses
# Campaign Contributions

Table standardization? Could I write a function that takes in a number of variables and creates the appropriate table?
I would need:
table name
a regex query to execute
a list of columns
IS THAT IT???
It seems like the rest of the process is pretty much identical from table to table - run the query, strip the gunk out, split the text, chunk the list, and build the dataframe